In [3]:
import numpy as np
import matplotlib.pyplot
from astropy.stats import LombScargle
from mpl_toolkits.mplot3d import Axes3D  #3d plotting
import time
import math
%matplotlib inline

# Topic 1 - Monte Carlo


## Monte Carlo Integration

To solve an integral $I =\int_a^b h(x)dx$, split $h(x)$ into $f(x)p(x)$, and the integral becomes an expectation value. Then $ I = (1/n) \sum_{i=1}^n f(x_i)$ where $x_i$ is drawn from $p(x)$.

## Random numbers

To sample a distribution randomly, one will need a uniform distribution. Let $u = CDF(x)$ then find the inverse to get $x = CDF^-1(u)$.

## Markov Chains

Markov chains follow the equation $\mathbf{X}_{n+1} = \mathbf{X}_{n} P$ where $P$ is the transition matrix. The rows must add to 1. Example:

$P = \begin{bmatrix} 0.9 & 0.1 \\ 0.5 & 0.5 \end{bmatrix} \quad X_0 = \begin{bmatrix} 1 & 0 \end{bmatrix} $

For Markov chain Monte Carlo: start somewhere, propose a move. If P(moved) > P(stay), accept. Otherwise accept with some probability $r$.

## Metropolis (Hastings)

The probability $r$ mentioned above may be accepted through two equations: 

Metropolis: $r = \frac{p(x^*)}{p(x)}$ where $x^*$ is the moved position.

Metropolis-Hastings $r = \frac{p(x^*)q(x|x^*)}{p(x) q(x^*|x)}$. $q(x^* | x)$ is the probability to move to $x^*$ given a starting position $x$. 

## Simulated Annealing

See both my implementation in lab1visuals as well as the lab1 question sheet for a run-down of how the algorithm works.

## Travelling Salesman

In the travelling salesman problem, there are different strategies to finding an optimal route:
1. The Greedy Method: Starting from a seed location, choose the nearest point next until the route needs to be closed.
2. The Brute Method: Determine every permutation of possible paths and then select the shortest
3. The Annealing Methods: Set up some initial path. Each step, swap two nodes, and accept if the route is shorted. Accept a longer route via the Metropolis algorithm. Continue for some time, keeping track of the shortest route. 

For permulations, import itertools, and use all = it.permutations(route)

In [ ]:
# ===============================================================
# ==================== RANDOM NUMBERS ===========================
# ===============================================================

def pseudorandom(I0, A, C, M, length=1000):
    ''' Generates a sequence of pseudorandom numbers using a remained method
    INPUT:
        I0     = seed for the sequence
        A      = number to multiply at each step
        C      = number to add at each step
        M      = modulous at each step
        length = how many numbers to generate'''
    seq = np.zeros(length)
    seq[0] = I0
    for j in range(1, length):
        seq[j] = int((A*seq[j-1] + C)%M)
    
    return seq

def accept_reject(num):
    '''Applies the accept / reject method to generate values x drawn from
    a distribution p
    INPUT:
        num:     the number of samples to return
    OUTPUT:
        samples: the list of x values drawn from p'''
    
    samples = []
    n=0
    while n < num:
        #get a random value from a probability f(x)
        test1 = f_sample()
        
        #get a random value between 0 and test1
        test2 = test1*np.random.random()
        
        #see if it is below the function if interest, and if so accept
        if test2 < p_interest(test1):
            n += 1
            samples.append(test1)
            
    return samples

#===============================================================
#======================== CHI-SQUARED ==========================
#===============================================================

def chi2_pdf(x,r=1):
    '''Returns the chi-squared distribution for some domain x
    INPUT:
        x - the domain on which to calculate the chi-squared pdf
        r - the degrees of freedom'''
    return (x**(r/(2-1))*e^(-x/2))/(2**(r/2)*math.gamma(x))
    

#================================================================
#==================== AUTOCORRELATION ===========================
#================================================================

def autocorrelation_wrap(data):
    ''' Calculates the autocorrelation function using wrapping around the data
    data - data set to apply auto-correlation to'''
    mean = np.mean(data)
    ac   = np.zeros_like(data)
    num  = np.zeros_like(data)
    den  = np.zeros_like(data)
    N    = data.size
    
    for k in range(N):
        for t in range(N):
            if not k == t:
                num[k] += (data[t] - mean)*(data[(t+k % N)] - mean)
                den[k] += (data[t] - mean)**2
    ac = num / den
    return ac

def autocorrelation(data):
    ''' Calculates the traditional auto-correlation function. DOES NOT WRAP AROUND
    This version is prone to dramatic swings at high k
    data - data set to apply auto-correlation to'''
    mean = np.mean(data)
    ac   = np.zeros_like(data)
    num  = np.zeros_like(data)
    den  = np.zeros_like(data)
    N    = data.size
    
    for k in range(N):
        for t in range(N-k):
            num[k] += (data[t] - mean)*(data[t+k] - mean)
            den[k] += (data[t] - mean)**2
    ac = num / den
    return ac

# ================================================================================
# =========================== MARKOV CHAINS ======================================
# ================================================================================

def markov(initial, p, steps):
    '''Given an initial probability distribution, predict some future state through
    Markov chains.
    INPUT:
        initial - the initial state
        p       - the probability matrix of changing states
        steps   - the number of time steps to simulate, including the initial
    OUTPUT:
        states - an array of all states the system proceeds through
        '''
    states = np.zeros(steps, initial.shape)
    states[0,] = initial
    
    for i in range(1,steps):
        initial = np.matmul(p, initial)
        states[i,] = initial

    return states
# ================================================================================
# ========================== SIMULATED ANNEALING =================================
# ================================================================================

# This code was all taken from Jack and Chris's presentation.

def box_muller(rand1,rand2,sigma,mu):
    '''Take in two uniform random number from [0,1].
    Make Gaussian using standard deviation sigma and shift mu.
    More info at: https://en.wikipedia.org/wiki/Box%E2%80%93Muller_transform'''
    
    z = np.sqrt(-2*np.log(rand1))*np.cos(2*np.pi*rand2)
    
    #now scale by standard deviation and shift
    z = z*sigma + mu
    
    return z

class sim_ann_multi:
    
    #same list of parameters
    def __init__(self,initial_temp,final_temp,temp_sched,initial_pos,hill,sweeps,sigma,minima=False,full_chain=False,boundary=None):
        ''' Initializes all variables for the method'''
        self.initial_temp = initial_temp
        self.final_temp = final_temp
        self.temp_sched = temp_sched
        self.initial_pos = initial_pos
        self.hill = hill
        self.sweeps = sweeps
        self.sigma = sigma
        self.minima = minima
        self.full_chain = full_chain
        self.boundary = boundary
        
        #other elements we need
        self.period = None
        self.zeros = None
        
        return
    
    #little bit of tweaking to get this to work properly.
    #Any dimension with a none zero period is given periodic boundary conditions
    def period_rap(self,pos):
        return np.where(np.not_equal(self.period,self.zeros),np.mod(pos-self.boundary[:,0],self.period)+self.boundary[:,0],pos)



    def run_sim_ann(self):
        #set the initial conditions
        temp = self.initial_temp
        pos = self.initial_pos
        
        #if we have periodic boundary conditions apply them
        if(not self.boundary is None):
            self.period = self.boundary[:,1]-self.boundary[:,0]
            self.zeros = np.zeros(self.initial_pos.size)
            pos = self.period_rap(pos)
            print(pos)
        
        current_cost = self.hill(pos)

        #if we are finding a minima then we need to flip this
        if(self.minima==True):
            current_cost*=-1

        #check if we want to record the full chain
        if(self.full_chain == True):
            chain = collections.deque([])
            trials = collections.deque([])
            chain.append(pos)
            trials.append(pos)
            

        #we will slowly reduce the temperature till the temperature scheduler stops it
        time = 0
        loop =True
        while loop:
            

            #do some sweeps at this temperature
            sweep_index = 0
            while sweep_index < self.sweeps:

                #choose a uniform random point 
                rand_pos = box_muller(np.random.rand(self.initial_pos.size),np.random.rand(self.initial_pos.size),self.sigma,pos)

                #if we have a boundary treat it as periodic and wrap around
                if(not self.boundary is None):
                    #loop the rand_pos back
                    rand_pos = self.period_rap(rand_pos)
                
                #let's compute the cost
                rand_cost = self.hill(rand_pos)

                #flip sign for minimization
                if(self.minima == True):
                    rand_cost  *=-1

                cost = rand_cost - current_cost

                #if this is a clear improvement then jump to that point
                if (cost>=0):
                    #accept change
                    pos = rand_pos
                    current_cost=rand_cost

                    #now try a jump with a temperature change chance
                elif(np.random.random()<np.exp(cost/temp)):
                    pos = rand_pos
                    current_cost=rand_cost

                #add the point to the deque
                if(self.full_chain==True):
                    chain.append(pos)
                    trials.append(rand_pos)

                #track number of sweeps
                sweep_index +=1

            #now we need to reduce the temperature
            temp,loop = self.temp_sched(self.initial_temp,self.final_temp,temp,time)
            time +=1

        #now return the final result

        #if we are asking for the chain then the chain
        if (self.full_chain==True):
            return np.asarray(chain),np.asarray(trials)

        return pos
    
def quick_plot(hill,fig_size=(8,4.5),title=""):
    %matplotlib inline
    
    fig = plt.figure(figsize=fig_size)
    ax = Axes3D(fig)
    
    hill_x = np.linspace(-3,3,500)
    hill_y = hill_x
    
    #plot the function
    hill_z =np.zeros((hill_x.size,hill_y.size))
    index_x=0
    while index_x <hill_x.size:
        index_y=0
        while index_y<hill_y.size:
            hill_z[index_y,index_x]= hill(np.array([hill_x[index_x],hill_y[index_y]]))
            index_y+=1
        index_x +=1
    
    hill_x, hill_y = np.meshgrid(hill_x,hill_y)
    
    ax.set_title(title)

    ax.plot_surface(hill_x,hill_y,hill_z,alpha=0.5,color='r')
    ax.plot_wireframe(hill_x, hill_y, hill_z, rcount=20, ccount=20, colors='k')

    #ax.set_legend(loc=legend_loc)
    plt.show(fig)
  
#linear for a constant decrease
def lin_temp_gen(steps):
    return lambda initial_temp,final_temp,temp,time : (initial_temp*(1-time/steps),not time == steps)

#geometric for a exponential decrease
def geom_temp_gen(steps):
    return lambda initial_temp,final_temp,temp,time : (np.max([initial_temp*(final_temp/initial_temp)**(time/steps),final_temp]),not time==steps)

f1 = lambda x : np.exp(-x[0]**2-x[1]**2)
f2 = lambda x : np.exp(-x[0]**2-x[1]**2) + 2*np.exp(-np.abs((x[0] - 1.7)**2 + (x[1]-1.7)**2))
f3 = lambda x : (1 - x[0])**2 + 100*(x[1] - x[0]**2)**2

quick_plot(f1,title="Function 1")
quick_plot(f2,title="Function 2")
quick_plot(f3,title="Function 3")

def find_maximum(function, temp_max, start ):
    f1_hill = sim_ann_multi(temp_max,0.000000001,geom_temp_gen(100),start,function,50,1)

    f1_results = np.zeros((150,2))

    for indx in range(150):
        f1_results[indx,:] = f1_hill.run_sim_ann()

    hist1, bin_edges1 = np.histogram(f1_results[:,0], bins=50)
    hist2, bin_edges2 = np.histogram(f1_results[:,1], bins=50)

    fig, ax = plt.subplots(1,2,sharey=True)
    ax[0].bar(bin_edges1[:-1], hist1, width = bin_edges1[1]-bin_edges1[0])
    ax[0].set_xlim(min(bin_edges1), max(bin_edges1))
    ax[0].set_xlabel('x coordinate')
    ax[0].set_ylabel('count')
    ax[0].set_title('Fuction 1 x maximum')

    ax[1].bar(bin_edges2[:-1], hist2, width = bin_edges2[1]-bin_edges2[0])
    ax[1].set_xlim(min(bin_edges2), max(bin_edges2))
    ax[1].set_xlabel('x coordinate')
    ax[1].set_title('Fuction 1 y maximum')

    fig.show
    print('Mean coordinates: ', f1_results[:,0].mean(), ",", f1_results[:,1].mean())
    print('Median coordinates: ', np.median(f1_results[:,0]), ",", np.median(f1_results[:,1]))
    
find_maximum(f1, 25, np.array([0,0]) )

# Topic 2 - Fourier Transform


## Series Coefficients

Trigometric series (Where T is the period over which the series is occurring:

$ f(t) = a_0 + \sum_{n=0}^{\infty} a_n \cos(n\omega t) + b_n \sin(n\omega_0 t) \quad \omega_0 = 2\pi / T$

$a_0 = \frac{1}{T} \int_{0}^T f(t) dt$

$a_n = \frac{2}{T} \int_{0}^T f(t) \cos(n\omega_0 t) dt$

$b_n = \frac{2}{T} \int_{0}^T f(t) \sin(n\omega_0 t) dt$

or $f(t) = \sum_{n=-\infty}^{+\infty} c_n e^{in\omega_0 t}$

$c_n = \frac{1}{T}\int_o^T f(t)e^{-in\omega_0 t} dt$

#### Symmetry:

Even functions have $a$ coefficients of 0. Odd functions have $b$ coefficients of 0.

#### Parseval's Theorem
Relates to power

$P = \frac{1}{T_0} \int_{-T_0/2}^{T_0/2} |x(t)|^2 dt = \sum_{k=-\infty}^{+\infty} |a_k|^2$

Another (in my opionion, better) way: $\int_{-\infty}^{\infty} |x(t)|^2 dt = \int_{-\infty}^{\infty} |\hat{x}(\omega)|^2 d\omega = P $

## Fourier Transform
Ouyed's preferred form:

$F(\omega) = \int_{-\infty}^{\infty} f(t) exp(-i\omega t)dt $

$f(t) = \frac{1}{2\pi} \int_{-\infty}^{\infty} F(\omega) \exp(i\omega t) d\omega$

Discrete:

$X_k = \sum_{0}^{N-1} x_n e^{\frac{-2\pi i kn}{N}} = W^{kn}*x_n$ 
The inverse is the same as above by analogy. 

#### Useful Transforms
Fourier transform of a dirac delta at zero is a constant. At any other frequency, it is a pure oscillation. The same is true vice versa. 

Notes FFT2 page 16 has a good summary. 

Convolution in one domain is multiplication in the other. 

Shift Theorem: $F(x(t-a)) = e^{2\pi i f a}F(x(t))$

## Using Numpy FFT

The basic function call: $\texttt{np.fft.fft(a, n=none, axis=-1, norm='None') }$

a - the data being used

n - what size the output fft should be. If none, is based off input size

axis - what axis to transform. If none given, use the last axis

norm - 'None' or 'ortho'  - none uses no normalization for the fft, $1/n$ for the inverse. 'Ortho' uses $1/\sqrt(n)$ for both.

## Inverse Numpy FFT

The basic call: $\texttt{np.fft.fft(a, n=none, axis=-1, norm='None') }$

Same as above.

The frequency output by iFFT is always between $(-5, 5)$. To turn into an actual frequency use the function written below

## Other Functions

$\texttt{np.fft.rfft(a, n=none, axis=-1, norm='None')}$  = computes only the positive frequencies of fft

$\texttt{np.fft.fft2(a, s=None, axes=(-2, -1), norm=None)}$ = 2d similar to fft using s as a multipidimesnional n, and using the last two axes

$\texttt{np.fft.fftshift(a, axes=None) }$ shifts fftfreq so the zero frequency is in the middle. Can specify which axes to shift

## Lomb-Scarlge

The Lomb-Scargle algorithm should be used if there are gaps in the data or if you want to look beyond the Nyquist limit. Astropy has a lomb-scargle function. Called at document start

Example: $\texttt{ls = LombScargle(t, count)}$

$frequency, power = ls.autopower(nyquist_factor=8, maximum_frequency=4, normalization='psd', method='chi2')$

These settings shown seemed to produce the best results. Normalization in particular is a problem unless set to sower spectral density. Chi2 seems to be the fastest legitimate Lomb-Scargle algorithm. Other methods may not be chi2. Or, see my prev. presentation slide for a hommage lomb-scargle. 

In [ ]:
# ==================================================================
# ========================= RUNGE-KUTTA ============================
# ==================================================================

from math import sqrt

def fx(x, y, z):
    
    return sig*(y - x)

def fy(x, y, z):
     return r*x - y - x*z
    
def fz(x, y, z):
    return x*y - b*z

def rk4(t0, x0, y0, z0, t_step, n):
    '''3D runge-kutta method function from Rosetta Code
    t0     = initial time
    x0     = initial x
    y0     = initial y
    z0     = initial z
    t_step = time interval between integration steps
    n      = number of time steps to take
    RETURNS the position and time results as separate lists
    '''
    
    dt = [0] * (n + 1)
    dx = [0] * (n + 1)
    dy = [0] * (n + 1)
    dz = [0] * (n + 1)
    h = t_step
    dt[0] = t = t0
    dx[0] = x = x0
    dy[0] = y = y0
    dz[0] = z = z0
    
    for i in range(1, n + 1):
        k1 = h * fx(x, y, z, r)
        l1 = h * fy(x, y, z, r)
        m1 = h * fz(x, y, z, r)
        
        k2 = h * fx(x + 0.5 * k1, y + 0.5 * l1, z + 0.5 * m1, r)
        l2 = h * fy(x + 0.5 * k1, y + 0.5 * l1, z + 0.5 * m1, r)
        m2 = h * fz(x + 0.5 * k1, y + 0.5 * l1, z + 0.5 * m1, r)
        
        k3 = h * fx(x + 0.5 * k2, y + 0.5 * l2, z + 0.5 * m2, r)
        l3 = h * fy(x + 0.5 * k2, y + 0.5 * l2, z + 0.5 * m2, r)
        m3 = h * fz(x + 0.5 * k2, y + 0.5 * l2, z + 0.5 * m2, r)
        
        k4 = h * fx(x + k3, y + k3, z + m3, r)
        l4 = h * fy(x + k3, y + k3, z + m3, r)
        m4 = h * fz(x + k3, y + k3, z + m3, r)
        
        dt[i] = t = t0 + i * h
        dx[i] = x = x + (k1 + k2 + k2 + k3 + k3 + k4) / 6
        dy[i] = y = y + (l1 + l2 + l2 + l3 + l3 + l4) / 6
        dz[i] = z = z + (m1 + m2 + m2 + m3 + m3 + m4) / 6
    return dt, dx, dy, dz

#===================================================================
#========================= Power Spectral Density===================
#===================================================================

def psd_ac(x, resolution=301):
    '''Computes the power spectral density of a data sequence using autocorrelation
    INPUT:
        x          = input data sequence
        resolution = number of frequencies omega to evaluate
    OUTPUT:
        s          = power spectral density
        omega      = frequencies at which s was computed'''
    
    omega = np.linspace(-np.pi, np.pi,resolution)
    s     = np.zeros_like(omega)
    ac    = autocorrelation_wrap(x)
    N     = ac.size
    k     = np.linspace(0,N,N)

    for i, val in enumerate(omega):
        s[i] = np.sum( ac * np.exp( -1j * val * k))
        
    return omega, s

#====================================================================
#============================ FFT TOOLS =============================
#====================================================================

def fftFreqFix(data, freq, t):
    ''' Takes np.fft.fftfreq frequency data and turns it into the appropriate frequency values for the spectrum
    INPUT:
        data = the data that was transformed
        freq = the output of fftFreq
        t    = the time spectrum associated with the data
    OUTPUT:
        freq = the re-calibrated frequency data'''
    
    T = t[-1] - t[0]
    df = 1/T
    N = data.size
    freq = freq*df*N
    
    return freq

#======================================================================
#========================= WINDOWS ====================================
#======================================================================

def hann_window(size):
    return 0.5 * (1 - np.cos(2 * np.pi * np.arange(0, size) / (size - 1)))

def blackmann_harris_window(size):
    t = 2 * np.pi * np.arange(0, size) / (size - 1)
    window = 0.35875 - 0.48829 * np.cos(t) + 0.14128 * np.cos(2 * t) \
             - 0.01168 * np.cos(3 * t)
    return window

# Topic 3 - Finite Differences


## Stencils

All stencils are using and example D.E.: $u_t = - u_x$

### Forward Euler
Find everything using central differences at the previous time step.

$u_i^{t+1} = u_i^t + \frac{\Delta t}{2\Delta x}(u_{i+1}^t - u_{i-1}^t)$

### Backward Euler
Find everything using central differences at the current time step (and a single point in the previous time step). Generally slower since it is an implicit method. Easy to code, but always unstable for certain freqs. via Von Neuman analysis.

$u_i^{t+1} = u_i^t + \frac{\Delta t}{2\Delta x}(u_{i+1}^{t+1} - u_{i-1}^{t+1})$

### Upwind
Essentially forward or backward differencing. Simulates the physics of moving information in a certain direction. Good for discontinuities, since it is less affected by shockwaves.

$u_i^{t+1} = u_i^t - \frac{\Delta t}{\Delta x}(u_{i+1}^t - u_{i}^t)$

### Crank-Nicolson
The average of forward and backward Euler. Computationally expensive, hard to code but always stable! Also one degree more accurate than backward Euler.

$u_i^{t+1} = u_i^t + \frac{\Delta t}{4\Delta x)}(u_{i+1}^t - u_{i-1}^t u_{i+1}^{t+1} - u_{i-1}^{t-1})$

### Leapfrog
Use a central difference method in time as well as space (go back 2 time steps). Better accuracy?

$u_i^{t+1} = u_i^t{t-1}+ \frac{\Delta t}{2\Delta x}(u_{i+1}^t - u_{i-1}^t)$

### Lax-Friedrichs
Interpolate the previous time step.

$u_i^{t+1} = \frac{1}{2}(u_{i+1}^t - u_{i-1}^t) + \frac{\Delta t}{2\Delta x}(u_{i+1}^t - u_{i-1}^t)$

### Lax
Uses an added diffusion term to smooth things out. Bad in that it changes the physics, good in that it has decent stability and smoothes things out. Better than forward Euler because it smooths out the frequencies that cause Forward euler to blow up.

$u_i^{t+1} = u_i^t - \frac{\Delta t}{2*2\Delta x}(u_{i+1}^t + u_{i-1}^t) + \frac{\Delta t}{2\Delta x}(u_{i+1}^t - u_{i-1}^t)$

### Lax-Wendroff
Goes to the next order in taylor series. Adds in artificial viscosity. Bad at sudden discontinuities.

$u_i^{t+1} = u_i^t - \frac{\Delta t}{2\Delta x}(u_{i+1}^t - u_{i-1}^t) + \frac{(\Delta t)^2}{2(\Delta x)^2}(u_{i+1}^t -2u_{i}^t + u_{i-1}^t)$

### 2-step Lax-Wendoff
Goes at half steps using conservative fluxes

Requires that the PDE be re-written in a conservative form: $\frac{\partial u}{\partial t} = - \frac{\Partial F(u)}{\partial x}$

$u_i^{t+1} = u_i^t - \frac{\Delta t}{\Delta x}(F_{i+1/2}^t - F_{i-1/2}^t)$

$F_{i\pm 1/2} = \frac{1}{2}(F(u_{i\pm 1}^t)+ F(u_i^t))$


### Jacobi

Used to simulate time independant problems by converges of a time dependent problem. Easier to understand than gauss-seidel. 

$\lim_{t \to \infty} \frac{\partial u}{\partial t} = 0$
so $u_i^{t+1} = u_i^t + \frac{\Delta t}{2\Delta x}(u_{i+1}^t - u_{i-1}^t)$
etc.

### Gauss-Seidel

Like jacobi, but two steps to maximize efficiency.


'Even cells': $u_i^{t+1} = u_i^t + \frac{\Delta t}{2\Delta x}(u_{i+1}^t - u_{i-1}^t)$

'Odd cells': $u_i^{t+1} = u_i^t + \frac{\Delta t}{2\Delta x}(u_{i+1}^{t+1} - u_{i-1}^{t+1})$

In [ ]:
# ==========================================================
# =================== STABILITY ============================
# ==========================================================

def cfl_1D(h, u):
    '''Determines the 1D cfl condition
    h - x resolution
    u - speed of the system (i.e. speed of light, sound)
    NOTE: May have to find the highest speed in the system'''
    courant = 0.5
    dt = courant * h / u
    
def cfl_2D(dx, dy, ux, uy):
    '''Determines the 2D cfl condition
    dx - x resolution
    dy - y resolution
    ux - speed of the system in the x direction
    uy - speed of the system in the y direction
    NOTE: ux**2 + uy**2 needs to equal speed of light, sound for subsonic medium
    NOTE: May have to find the highest speed in the system'''
    courant = 0.5
    dt = courant / (ux/dx + uy/dy)
    return dt

# =================================================================
# =======================  RELAXATION =============================
# =================================================================

def stencil(u, i, j):
    '''Needs to be re-written for each problem'''
    return u[i,j]

def relaxation(u, xx, yy):
    '''Solves a capacitor system using the Jacobi method
    u - variable matrix - assumes a 1 pixel ghost zone on all sides
    xx - x coordinates made from meshgrid
    yy - y coordinates made from meshgrid'''
    
    L = xx[:,-2]
    Nx, Ny = u.shape
    Nx = Nx - 2       #Take ghost zone into account
    Ny = Ny - 2       #Take ghost zone into account
    x_res = xx[:,1] - xx[:,0]
    y_res = yy[1,:] - yy[0,:]
    
    dt = cfl(x_res)  #May need to change to 2D cfl
    u_new = np.zeros_like(u)
    
    tiny = 1.0*10**(-8)          #Convergence criterion
    for t in range(10000): 
        u = boundary(u,xx,L)     #Needs to be written for each problem
        for j in range(1,N+1):
            for i in range(1,N+1):
                # Need to change stencil for different method / D.E.
                u_new[i,j] = stencil()
       
        diff = np.amax(np.abs(u_new[1:-1,1:-1] - u[1:-1,1:-1]))
        #Exit the loop when the system converges
        if diff < tiny:
            break
        u[:,:] = u_new
        
    return u

def diagonal_form(a, upper = 1, lower= 1):
    """
    a is a numpy square matrix
    this function converts a square matrix to diagonal ordered form
    returned matrix in ab shape which can be used directly for scipy.linalg.solve_banded
    
    This has been taken from: https://github.com/scipy/scipy/issues/8362
    """
    n = a.shape[1]
    assert(np.all(a.shape ==(n,n)))
    
    ab = np.zeros((2*n-1, n))
    
    for i in range(n):
        ab[i,(n-1)-i:] = np.diagonal(a,(n-1)-i)
        
    for i in range(n-1): 
        ab[(2*n-2)-i,:i+1] = np.diagonal(a,i-(n-1))

    mid_row_inx = int(ab.shape[0]/2)
    upper_rows = [mid_row_inx - i for i in range(1, upper+1)]
    upper_rows.reverse()
    upper_rows.append(mid_row_inx)
    lower_rows = [mid_row_inx + i for i in range(1, lower+1)]
    keep_rows = upper_rows+lower_rows
    ab = ab[keep_rows,:]


    return ab

def direct(N, xres, alpha):
    ''' solves a relaxation method problem using the direct method
    N - size of active zone - assuming active zone is square
    xres - resolution of the x grid in the active zone'''
    #Set up the matrices
    sq = (N+2)**2
    A = np.zeros((sq,sq))
    b = np.zeros(sq)
    x = np.zeros_like(b)
    u = np.zeros((N,N))
    
    c = -(4 + alpha*x_res**2)   #This will need to change based on stencil
    
    for i in range(N+2):        #1st u index
        for j in range(N+2):    #2nd u index
            #This section requires the use of indices. For a variable u_i,j in the active zone,
            #The index on b is (N+2)*i + j. The location of stencil values on A can be found
            #by substituting (i+1) etc. into the above expression. 
            count = (N+2)*i + j 
            
            #Assign the values for boundary zones
            if (i == 0) or (j == 0) or (i == N+1) or (j == N+1):
                A[count, (N+2)*i+j]     = 1
                b[count]   = 1
            #Assign the stencil coefficients
            else:
                A[count, (N+2)*(i+1)+j] = 1
                A[count, (N+2)*(i-1)+j] = 1
                A[count, (N+2)*i+(j+1)] = 1
                A[count, (N+2)*i+(j-1)] = 1
                A[count, (N+2)*i+j]     = c
                b[count]                = 0
                
    #need to use special method for banded matrix. Using the scipy solve_banded system.
    #Requires that the matrix be re-written in a better form up: # of diags above, lo: num of diags below
    up = N+2
    lo = N+2
    ab = diagonal_form(A, up, lo)
    x = sl.solve_banded((lo,up), ab, b)
    
    #Need to turn x back into a 2d array:
    u = x.reshape((N+2,N+2))
    
    return u

# ===========================================================================
# ============================ CONSERVATIVE FORMS ===========================
# ===========================================================================

def flux(j, u_on, u_off, v, h, half):
    """ Computes the viscous burger flux at a given time
    curr  = the current index being considered
    u_on  = the u values on the same grid as the flux being considered
    u_off = the u values on the grid a half step off the current flux grid being considered
    v     = the viscosity - CONSERVATIVE IF V=0
    half  = bool for if the current grid being considered is on the half step or not
    h     = the x resolution of the grids
    f     = the flux at the curr location"""
    #note: the delta x between the half steps is half the defined resolution - cancels out the factor 2 from the CFD
    if half:
        f = 0.5*u_on[j]**2 - (v/h)*(u_off[j+1] - u_off[j])
    else:
        f = 0.5*u_on[j]**2 - (v/h)*(u_off[j] - u_off[j-1])
    return f

def initial(u, x):
    '''The initial conditions for a conservative form system
    u - the variable grid
    x - the x coordinates of the variable grid'''
    for i, val in enumerate(x):
        u[i] = -np.sin(np.pi*val)
    return u

def lw2step(u, u_half, f, f_half, v, h, x, conserve, filename):
    '''Solves a conservation equation using the Lax-wendoff 2-step method. Assumes 
    u, f, u_half, x_half initial conditions have already been established.
    u -      the variable grid
    u_half - the variable grid one half time step ahead of u
    f -      the flux '''
    dt = 0.1
    time_steps = 20
    
    #Writes the data to a file for the initial conditions
    file = open(filename+'%03d'%0+'.txt', 'w')
    file.write("time = %10.6f"%0.0+'\n')
    for i in range(1,N+1):
        file.write("%12.8f"%x[i])
        file.write("%12.8f"%u[i])
        file.write('\n')
    file.close()
    
    u_new      = np.zeros_like(u)
    u_half_new = np.zeros_like(u_half)
    f_new      = np.zeros_like(f)
    f_half_new = np.zeros_like(f_half)
    
    for n in range(time_steps):
        #Solve for u and flux at a half-step forward in time
        for j, val in enumerate(u_half):
            u_half_new[j] = 0.5*(u[j+1] + u[j]) - (dt/(2*h))*(f[j+1] - f[j])
            
        for j, val in enumerate(f_half):
            if conserve == True:
                f_half_new[j] = flux_conserve(j,u)
            else:
                f_half_new[j] = flux(j, u_half, u, v, h, half=True)
        
        #Solve for u and flux at the next full step in time
        for j in range(1,N+1):
            u_new[j] = u[j] - (dt/h)*(f_half_new[j] - f_half_new[j-1])
            
        u_new[0]  = 0 #reset boundary conditions
        u_new[-1] = 0
        
        for j in range(1,N+1):
            if conserve == True:
                f_new = flux_conserve(j,u_new)
            else:
                f_new[j] = flux(j, u_new, u_half_new, v, h, half=False)
        
        #Advance dummy variables for potential next loop
        u[:] = u_new
        u_half[:] = u_half_new
        f[:] = f_new
        f_half[:] = f_half_new
        
        #write the results to a new file
        time = (n+1)*dt
        file = open(filename+'%03d'%(n+1)+'.txt', 'w')
        file.write("time = %10.6f"%time+'\n')
        for i in range(1,N+1):
            file.write("%12.8f"%x[i])
            file.write("%12.8f"%u[i])
            file.write('\n')
        file.close()

# Appendix 1: Visualization Code


## Histogram Note

Matplot lib will plot directly with data. Numpy will produce histogram data, but it will need to be plotted using pyplot's $\texttt{plt.bar()}$ function. Numpy histogram works as $\texttt{np.histogram(a, bins=10, range=None, normed=False, weights=None, density=None)}$

a - the data to be visualized

bins - how many subdivisions to make of the data

range - what are the lower and upper bounds for the histogram. Can auto-set to max and min values

normed - ignore

weights - weighting to give each cell. Overridden by density

density

In [ ]:
#=============================================================
#============== SETTING DEFAULT FONT SIZE ====================
#=============================================================

plt.rc('axes', labelsize=18)
plt.rc('font', size=12)
plt.rc('xtick', labelsize=12)    # fontsize of the tick labels
plt.rc('ytick', labelsize=12)    # fontsize of the tick labels
plt.rc('legend', fontsize=14)
plt.rc('figure', titlesize=12)  # fontsize of the figure title

#==============================================================
#======================== HISTOGRAMS ==========================
#==============================================================

# Example:
plt.figure(0)
n, locs, s = plt.hist(tau, bins=20)
#n - # of counts in each bin
#locs - the leftmost edge value of each bin
# s  - ignore this
#tau - the data to visualize
#bins - the number of bins to divide the data into

#plotting an expected function overtop
x = np.linspace(0,10,99)
y = np.exp(-1*x)*np.max(n)
plt.plot(x,y)
plt.xlabel(r'$\tau$')
plt.ylabel(r'$P(\tau)$')
plt.savefig('fig6.png')

#Using numpy example
hist_sum, bin_edges = np.histogram(rand_sum, bins=100)
plt.bar(bin_edges, hist_sum)

# Appendix 2: Formatting

To format strings, use % in the string, followed by any leading zeros, followed by the width of the space, followed by any decimals, followed by the number of decimal places.

| Letter | Type     |
|--------|----------|
|   b    | Binary   |
|   c    |Character |
|   d    | Decimal  |
|   f    | Float    |
|   s    | String   |

Example:

```python
"This is a %006f.2"%variable" 
```

Yields something like: This is a 003.14


In [2]:
variable = 3.14159

print("This is a %006.2f"%variable)

This is a 03.14


# Appendix 3: Other tools


$\texttt{ np.polyfit(x, y, degree) }$ fits a polynomial of specified degree to the data

In [ ]:
# ======================================================================
# ===================== READING FILES ==================================
# ======================================================================

#Example
file = open('phys581-binary.txt', 'r') #filename, mode

t = []
count = []

for indx in range(0,28400):   #Need to get the line count from notepad++
    line = file.readline()
    line = line[:-1]          #Removes endlines
    t.append(float(line[:11]))
    count.append(float(line[11:]))